In [ ]:
import numpy as np
from scipy.optimize import approx_fprime
from scipy.linalg import eigh

# Define constants
TX1_Bandwidth = 10  # Example value, adjust as necessary
RX1_Noise_Power = 1  # Example value, adjust as necessary
B_max = 10  # Example value, adjust as necessary
P_max = 10  # Example value, adjust as necessary
beta = 1  # Example value, adjust as necessary
a1_t = 1  # Example value, adjust as necessary

def objective(x, a1_t):
    x1, x2, x3, x4 = x
    xi_1 = TX1_Bandwidth * np.log2(1 + x4 / RX1_Noise_Power)
    xi_2 = TX1_Bandwidth * np.log2(1 + B_max / RX1_Noise_Power)
    xi_3 = P_max * abs(x4)
    xi_4 = 2 * xi_2 * x3 + beta * abs(B_max)
    W = TX1_Bandwidth
    J = -2 * W**2 * x1**2 * (np.log2(1 + xi_1 * x4))**2 - 2 * W * a1_t * x1 * np.log2(1 + xi_1 * x4) - 2 * xi_2**2 * x3**2 - xi_4 * x3 - beta * x1 * x4
    return J

def hessian_approx(f, x0, epsilon=1e-5, *args):
    n = len(x0)
    hessian = np.zeros((n, n))
    step = np.zeros(n)
    for i in range(n):
        for j in range(n):
            if i == j:
                step[i] = epsilon
                f1 = f(x0 + step, *args)
                f2 = f(x0 - step, *args)
                f3 = f(x0, *args)
                hessian[i, j] = (f1 - 2*f3 + f2) / (epsilon**2)
                step[i] = 0
            else:
                step[i] = epsilon / 2
                step[j] = epsilon / 2
                f1 = f(x0 + step, *args)
                f2 = f(x0 - step, *args)
                step[j] = -epsilon / 2
                f3 = f(x0 + step, *args)
                step[i] = -epsilon / 2
                f4 = f(x0 + step, *args)
                hessian[i, j] = (f1 - f2 - f3 + f4) / (epsilon**2)
                step[i] = 0
                step[j] = 0
    return hessian

# Initial guess
x0 = np.array([0.1, 0.1, 0.1, 0.1])

# Calculate the Hessian
hess = hessian_approx(objective, x0, 1e-5, a1_t)

# Check if the Hessian is positive semidefinite
eigenvalues = eigh(hess, eigvals_only=True)

print("Hessian matrix at x0:")
print(hess)
print("Eigenvalues of the Hessian:")
print(eigenvalues)
print("Is the Hessian positive semidefinite?:", np.all(eigenvalues >= 0))
